<div id="bg-slide">
    <center>
    <h1>Basis Sets (PWs and GTOs)</h1>
<center>
<br>
<table>
  <tr>
      <td><p><h1>Team Rutgers</h1></p><p><h2>(Michele, Ezekiel & Valeria)</h2></p><p><h2>Rutgers University-Newark</h2></p></td> 
      <td><img src="./figures/logos/ASESMA_logo.png" width=300 height=300 /></td>
  </tr>
  <tr>
    <td></td>
    <td> https://prg.rutgers.edu</td>
  </tr>
</table>
<br>
<h3>Retrieve this presentation at:</h3>
<br>
<center>https://github.com/Quantum-MultiScale-Template/DFT-Intro/2025/Basis_Sets.ipynb</center>

<br>

<center><h3> ASESMA 2025 -- Accra, Ghana -- June 10, 2025</h3></center>
</div>


# Split into `N` groups

 - Assign number `1-N` to each student
 - Groups sit together
 - Possibly have 1 instructor per group

In [3]:
from IPython.display import IFrame
import numpy as np
import matplotlib.pyplot as plt
from ase.build import molecule
from pyscf import gto

# My assumptions

 - Basic linear algebra (vectors, matrices, operators, wavefunctions)
 - Undergraduate QM
 - Dirac notation (i.e., i-th vector $\to |i\rangle$ or $|\psi_i\rangle$, scalar product between <i>i</i> and <i>j</i> $\to \langle \psi_i | \psi_j\rangle$)
 - Kohn-Sham equations

# Why are basis sets needed?

 - They provide a representation
 - Allow us to discretize the problem

# Let's expand a function in a basis

$$
\psi_k(r) = \sum_i C_{ik} \chi_i(r)
$$

 - $\{\chi_i(r)\}$ are basis functions forming a <b>basis set</b>
 - $\{C_{ik}\}$ are expansion coefficients, $C_{ik}\in \mathbb{C}$

# Challenge 1
- Provide one example of basis functions and explain why they are useful

# Examples of basis sets

 - Plane waves (PW)
$$
 \chi_{G}(r) = \frac{1}{\sqrt{\Omega}} e^{iGr}
$$

 - Gaussian-Type Orbitals (GTOs)
 $$
 \chi_{\mu,\alpha}(r) = \left(\frac{2\alpha}{\pi}\right)^{d/2} e^{-\alpha |r-\mu|^2}
 $$

 - Numerical Orbitals (Javier will introduce later this week)
 - ...

# What do they look like? Let's use `Python`

In [ ]:
L=10; X = np.linspace(start=0,stop=L,num=1000,endpoint=False)
def pw(x,G):
    return 1.0/np.sqrt(L)*np.exp(1j*G*x)
def gto(x,center,alpha):
    return np.sqrt(alpha/np.pi)*np.exp(-alpha*(x-center)**2)

In [ ]:
fig = plt.figure(figsize=(10,6))
#plt.plot(X,np.real(pw(X,G=1)), label=r"Re of PW with $G=1$")
#plt.plot(X,np.imag(pw(X,G=1)), label=r"Im of PW with $G=1$")
plt.plot(X,gto(X,center=5,alpha=1),label=r"GTO centered in $x=5$, with $\alpha=1$")
plt.ylim([-.5,1]); plt.legend(); #plt.plot(5, 0, 'ko', markersize=20);

# Challenge 2
 - Discretize the KS equations using a basis set $\{\chi_i(r)\}$ knowing you can write the KS orbitals as $\psi_k(r) = \sum_i C_{ik} \chi_i(r)$ and the KS Hamiltonian matrix elements are $H_{ij}=\langle \chi_i | \hat H | \chi_j \rangle$ and the overlap elements among the basis functions are $S_{ij}=\langle \chi_i|\chi_j\rangle$.
 - The final result should be 
 $$\sum_i H_{ji}C_{ik} = \varepsilon_k \sum_i C_{ik}S_{ji} ~\text{ or in matrix form }~ \mathbb{HC}=\mathbb{SC\varepsilon}$$ 

# Challenge 3
 - Are GTOs an orthonormal basis set? Justify.
 - Are PWs an orthonormal basis set? Justify.
 
 The condition for orthonormality $S_{ij}=\delta_{ij}$.

# Size of basis sets

 - To be useful, basis sets should have reduced size,
    $$ \psi_k(r) = \sum_{i=1}^{M} C_{ik} \chi_i(r), ~ \text{where}~M<+\infty$$

 - For PW basis, this usually means imposing a cutoff on the maximum value of $G$
 

 - For GTO basis, the size is determined by the basis set developer. Several basis set sizes are available.
 

# Challenge 4
This is to help you breeze through what we will do next.
 - Find the wavefunctions that solve the time-independent Schrödinger equation of a single particle of mass $m=1$au in a periodic 1D box of size $L$. Use atomic units ($\hbar = 1$) throughout.
 - Remember, with PBCs the continuity conditions are $\psi(x+L)=\psi(x)$.

# PW cutoff in 1d - let's use FFT

 - First, let's discretize a 1d box of length $L=10$ au into $N$ points, $\{x_k=\frac{k}{N}L\}$
<center><img src="./figures/science/line.png" width=400 /></center>

In [ ]:
N=7; L=10; x = np.linspace(start=0,stop=L,num=N, endpoint=False); x

 - Second, we generate all the available $G$ points according to <a href="https://numpy.org/doc/stable/reference/routines.fft.html">FFT</a>: $~e^{iG_j x_k}=e^{i\left(\frac{2\pi j}{L}\right) \left(\frac{k}{N}L\right)}=e^{i2\pi \frac{jk}{N}}$

 - Thus: as $x_k=\frac{k}{N}L$, it must be that $G_j = \frac{2\pi j}{L}$ with $j=0,\pm 1, \pm 2,\ldots \pm \frac{N}{2}$
<center><img src="./figures/science/lineg.png" width=400 /></center>

In [ ]:
G = (2*np.pi)*np.fft.fftfreq(N,d=L/N); G

<center>
    <div class="alert alert-success">
        Note: higher $N$ &rarr; higher maximum value of $G$
    </div>
</center>

# Effect of a PW cutoff

Now compute the Fourier transform of $f(x)$, $\tilde f(G)$

$$
\tilde f(G) = \int dx f(x) e^{-iG x} \simeq \frac{L}{N}\sum_{i=0}^{N-1} f_i e^{-i G x_i}
$$

In [ ]:
f = gto(x,center=5,alpha=1)

tilde_f = np.fft.fft(f)*L/N

Now compute the inverseFourier transform on a new domain (not the original N points)

$$
f(x) = \frac{1}{\Omega}\int dG \tilde f(G) e^{i G x} \sim \frac{1}{L}\sum_{i=0}^{N-1} \tilde f_i e^{i G_i
x}
$$

In [ ]:
def gto_in_pw(fft,G,X):
    N=len(G); f_X = np.zeros_like(X,dtype=np.complex128)
    for i in range(N):
        f_X += fft[i]*np.exp(1j*G[i]*X)  
    return np.real(f_X)/L

# Plot of the effect of a PW cutoff

In [ ]:
plt.plot(X,gto(X,center=5,alpha=1),label=r"Exact $f(x)$")
plt.plot(X,gto_in_pw(tilde_f,G,X),label=r"Approximate $f(x)$")
plt.legend()

# Challenge 5
 - What is an appropriate number of PWs for representing $f(x)$? Discuss within the group and provide a quantitative justification.

# The major drawback of PWs
- Need many PWs to represent "spiky" or highly oscillatory features
- Thus, **pseudopotentials** are typically employed to
    1) avoid the *Coulomb singularity* of the electron-nuclear interaction potential
    2) avoid *core of atoms* (spiky/oscillatory density and orbitals, more from Javier tomorrow!)

# Why are PW basis so useful?

 - Orthonormal! 

 - Systematically improvable: higher $N$ (or equivalently $\max\left[ G \right]$) leads to better results. 

 - Evaluation of the kinetic energy operator and solution of the Poisson equation are <b>easy</b>

# Matrix elements of the kinetic energy operator, needed to solve $\mathbb{HC=C\varepsilon}$

$$
\langle \chi_i | \hat T | \chi_j \rangle = -\frac{1}{2\Omega} \int dx  (-G_j^2)  e^{i(G_j-G_i)}x = \mathbf{\color{red}{\frac{G_j^2}{2}\delta_{ij}}}
$$

<center> It's diagonal!</center>

# Classical Coulomb (Hartree) interaction energy and potential

$$
v_H(r) = \int dr' \frac{n(r')}{|r-r'|} \equiv n \star \frac{1}{|r|}
$$

<center>
    <div class="alert alert-success">
        Let's use convolution theorem: $f = h \star g \to \tilde f = \tilde h \cdot \tilde g$
    </div>
</center>

$$
\tilde v_H(G) = \tilde n(G) \frac{4\pi}{G^2} \to v_H(r) = \frac{1}{\Omega}\int \tilde v_H(G) e^{iGr}dG = \frac{1}{\Omega}\int \tilde n(G) \frac{4\pi}{G^2} e^{iGr}dG
$$

<center>
    <div class="alert alert-success">
        <b>Thanks to FFT, the potential is computed with a $\mathcal{O}(N\ln N)$ number of operations!
    </div>
</center>

# Matrix elements of $v_s(r)$, needed to solve $\mathbb{HC=C\varepsilon}$

$$
\langle \chi_i | v_s | \chi_j\rangle = \frac{1}{\Omega}\int dr v_s(r) e^{i(G_j-G_i)r} 
$$

<center>
    <div class="alert alert-danger">
        <b>Not a diagonal matrix</b>
    </div>
</center>

But... we can use the $G$ components of $\tilde v_s(G)$:
$$
\langle \chi_i | v_s | \chi_j\rangle = \frac{1}{\Omega}\tilde v_s(G_i-G_j)
$$
which gives the matrix elements of $v_s$ <b>almost</b> directly.

# Summary of PW basis sets

- PWs are orthonormal and systematically improvable basis sets
- PWs are handled computationally with Fast Fourier Transforms which scale like $\mathcal{O}(N\ln N)$ rather than quadratically
- Coulomb problem approached with convolution theorem and FFTs
- Matrix elements of the KS Hamiltonian are easily found

# A short overview of GTO basis sets

- GTOs are not orthogonal

- GTOs are atom-centered. When atoms *move*, the basis set *changes*

 - $N_{GTO}<<N_{PW}$

 - The Coulomb problem scales like $N^2$ for small to medium sizes systems

# Densities and potentials in GTOs

The electron density is represented as:

$$
n(r) = \sum_{\mu\nu} P_{\mu\nu} \chi_\mu(r)\chi_\nu(r)
$$

The Hartree energy is given by

$$
E_H[n] = \frac{1}{2}\int \frac{n(r)n(r')}{|r-r'|}drdr' = \frac{1}{2} \sum_{\mu\nu\sigma\tau} P_{\mu\nu} P_{\sigma\tau} \int \chi_\mu(r)\chi_\nu(r) \frac{1}{|r-r'|}\chi_\mu(r')\chi_\nu(r')drdr'
$$

The 4-index matrix elements 

$$
\int \chi_\mu(r)\chi_\nu(r) \frac{1}{|r-r'|}\chi_\mu(r')\chi_\nu(r')drdr' = \left(\mu\nu|\sigma\tau\right) \to E_H = \frac{1}{2}\sum_{\mu\nu\sigma\tau} P_{\mu\nu} P_{\sigma\tau}\left(\mu\nu|\sigma\tau\right)
$$

are called **electron repulsion integrals** or **ERI**.

# Challenge 6
 - Derive the matrix elements of the Hartree potential, $\langle \mu | v_H | \nu \rangle$ and show that they require an $N^2$ number of operations.

# A glance at existing basis sets

In [ ]:
IFrame(src="https://www.basissetexchange.org/", width="100%", height="600")

# How many GTOs are needed?

 - It depends on the system. Let's check water...

In [16]:
mol = gto.Mole(); mol.build(atom = '''O 0 0 0; H  0 1 0; H 0 0 1''', basis = '6-311g*'); mol.nao

24

In [17]:
mol.ao_labels()

['0 O 1s    ',
 '0 O 2s    ',
 '0 O 3s    ',
 '0 O 4s    ',
 '0 O 2px   ',
 '0 O 2py   ',
 '0 O 2pz   ',
 '0 O 3px   ',
 '0 O 3py   ',
 '0 O 3pz   ',
 '0 O 4px   ',
 '0 O 4py   ',
 '0 O 4pz   ',
 '0 O 3dxy  ',
 '0 O 3dyz  ',
 '0 O 3dz^2 ',
 '0 O 3dxz  ',
 '0 O 3dx2-y2',
 '1 H 1s    ',
 '1 H 2s    ',
 '1 H 3s    ',
 '2 H 1s    ',
 '2 H 2s    ',
 '2 H 3s    ']

# We could go on forever with `basis sets`...